In [0]:
import dlt
import json
from pyspark.sql.types import *
from pyspark.sql import functions as f

In [0]:
# x = dbutils.secrets.get(scope="akv-dlp", key="eventhub-sqlservercdc-access-key")
# x[:14]

In [0]:
def dummy_df():
    data = [["A",100],["B",200]]
    schema = StructType([
        StructField("key", StringType(), True),
        StructField("value", StringType(), True)
    ])
    df = spark.createDataFrame(data, schema)
    return df

In [0]:
import json
source_to_bronze_config_path = "../../Config/SourceToBronze.json"  # Path to your JSON file
with open(source_to_bronze_config_path, 'r') as file:
    config = json.load(file)

In [0]:
eventhub_sources = [item for item in config["sources"] if item["source_format"] == "eventhub"]

# @dlt.table(name = "silver.ssff")
# def table():
#     return dummy_df()

for source in eventhub_sources:
    @dlt.table(name = f"silver.{source['table_name']}")
    def table():
        return dummy_df()


# for source in eventhub_sources:
#     KAFKA_OPTIONS = return_eventhub_config(source["eventhub_namespace"], source["eventhub"])
#     @dlt_table(name=f"bronze_{source['table_name']}")
    


In [0]:
# df = spark.read \
#     .format("kafka") \
#     .options(**KAFKA_OPTIONS) \
#     .load() \
#     .withColumn("records", f.col("value").cast("string"))

In [0]:
# @dlt.table(
#   comment="CDC events from SQL Server",
#   partition_cols=["event_time"]
# )
# def sqlservercdc():

In [0]:
# import json
# source_to_bronze_config_path = "../../Config/SourceToBronze.json"  # Path to your JSON file
# with open(source_to_bronze_config_path, 'r') as file:
#     config = json.load(file)

In [0]:
# eventhub_sources = [item for item in config["sources"] if item["source_format"] == "eventhub"]

In [0]:
# %sql
# USE CATALOG dlp_dev

In [0]:
# %sql
# CREATE SCHEMA bronze MANAGED LOCATION 'abfss://dataplatform@dataplatformpocaytc1028.dfs.core.windows.net/bronze'

In [0]:
# %sql
# CREATE SCHEMA silver MANAGED LOCATION 'abfss://dataplatform@dataplatformpocaytc1028.dfs.core.windows.net/silver'

In [0]:
# %sql
# DROP SCHEMA IF EXISTS bronze CASCADE

In [0]:
# df = spark.read \
#     .format("kafka") \
#     .options(**KAFKA_OPTIONS) \
#     .load() \
#     .withColumn("records", f.col("value").cast("string"))

In [0]:
# def parse(df):
#   return (df
#     .withColumn("records", col("value").cast("string"))
#     .withColumn("parsed_records", from_json(col("records"), payload_schema))
#     .withColumn("iot_event_timestamp", expr("cast(from_unixtime(parsed_records.timestamp / 1000) as timestamp)"))
#     .withColumn("eh_enqueued_timestamp", expr("timestamp"))
#     .withColumn("eh_enqueued_date", expr("to_date(timestamp)"))
#     .withColumn("etl_processed_timestamp", col("current_timestamp"))
#     .withColumn("etl_rec_uuid", expr("uuid()"))
#     .drop("records", "value", "key")
#   )

In [0]:
# df.write.mode("overwrite").saveAsTable("silver.dummy")